# Linear Model Selection and Regularization


The materials used in this tutorial are based on the applied exercises provided in the book "An Introduction to Statistical Learning with Applications in R" (ISLR). We are trying to demonstrate how to implement the following methods of selecting the most appropriate model that explains the data:
    * Subset selection
    * Shrinkage method
        * Ridge regression (L-2 regularization)
        * Lasso (L-1 regularization)

Even though you will learn how to use the above methods, it still is worth trying by yourself
    1. Sections 6.5 and 6.6
    1. The other practical exercises in section 6.8 (optional)
    
The libraries that you needed are
* <a href="https://cran.r-project.org/web/packages/leaps/index.html">leaps</a>
* <a href="https://cran.r-project.org/web/packages/glmnet/index.html">glmnet</a>
    
## 1. Subset Selection, Stepwise Selection and Lasso for selection features

Subset selection is an approach of identifying a subset of predictors (or attributes) that we believe have strong an association with the response variable. In this exercise, we will generate simulated data, and will use this data to perform best subset selection.

### 1.1 Generate simulated data
Here, we are going to use the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Normal.html">rnorm()</a> function to generate a predictor $X$ of length $n = 100$, as well as a noise vector of length $n = 100$. Note that we start with setting the seed for the random number generate, the purpose of which is to make our analysis reproducible. 

In [ ]:
set.seed(1)

We first generate 100 random values that are normally distributed:

Then, generate some random noises as irreducible errors.

Now, we can generate a response vector $Y$ of length $n = 100$ according to the model 
    $Y = \beta_0+\beta_1 X +\beta_2 X^2 +\beta_3 X^3 + \epsilon $ 
where $\beta_0$, $\beta_1$, $\beta_2$ and $\beta_3$  are constants of your choice.
Here, we will set those coefficients as follows.

In [ ]:
b0 <- 2
b1 <- 3
b2 <- -1
b3 <- 0.5

Given the coefficients, it is not hard to generate the simulated data using the polynomial function.

In [ ]:
y <- b0 + b1 * x + b2 * x^2 + b3 * x^3 + eps

### 1.2  Perform best subset selection 
In this task, we will use the regsubsets() function (refer to the documentation of <a href="https://cran.r-project.org/web/packages/leaps/leaps.pdf">leaps</a> for the function specification) to perform best subset selection in order to choose the best model containing the predictors $X$, $X^2$, $\dots$, $X^{10}$. What is the best model obtained according to $C_p$, $BIC$, and adjusted $R^2$? Show some plots to provide evidence for your answer, and report the coefficients of the best model obtained. Note you will need to use the <a href="https://stat.ethz.ch/R-manual/R-devel/library/base/html/data.frame.html">data.frame()</a> function to create a single data set containing both $X$ and $Y$.

We start with loading the library,

In [ ]:
library(leaps)

Then, put our data into a data frame,

In [ ]:
data.full <- data.frame(y = y, x = x)
data.full

Now, let's have a look at our data.

In [ ]:
plot(data.full$x, data.full$y)

It seems that the plot shows the relationship between $X$ and $Y$ is linear. However, it is indeed not, as our data were generated from a polynomial function with degree 3. Our goal here is to find the best model contains at most 10 variables, from $X$ to $X^{10}$. The first thing that you need to do is to call the <font color="blue"> regsubsets()</font> function. To see the documentation of this function, simply type

In [ ]:
help(regsubsets)

The above documentation tells us that the first argument is the model formula for the full model, which is
$\hat{Y} = \beta_0 + \beta_1 X + \beta_2 X^2 + \dots + \beta_{10} x^{10}$. We should also specify the data and the maximum size of subset to examine, which is 10 here.

In [ ]:
regfit.full <- 

The summary() function will output the best set of variables for each model size from 1 to 10.

In [ ]:
reg.summary <- summary(regfit.full)
reg.summary

An asterisk indicates that a variable in the corresponding column is included in the model. For example, the best one-variable model only contains $X$and  the best three-variable model contains $X$, $X^2$ and $X^5$. As discussed in section 6.5.1 in the text book, ISLR, the <font color="orange">nvmax</font> option can be used in order to return as many as variables as are desired, subject to $nvmax <= p$. Besides, what has been show above, the <font color="blue">summary()</font> function also returns:

In [1]:
names(reg.summary)

ERROR: Error in eval(expr, envir, enclos): object 'reg.summary' not found


We can examine these outputs to identify the best overall model. First, let's find the best overall model using Mallow's CP.

In [ ]:
plot(reg.summary$cp, xlab = "Number of variables", ylab = "C_p", type = "l")
mincp = which.min(reg.summary$cp)
points(mincp, reg.summary$cp[mincp], col = "red", cex = 2, pch = 20)
mincp

The plot suggests that the best overall model is a model with 3 variables, what are the three variables and their coefficients?

In [ ]:
coef(regfit.full, mincp)

Similarly, we can use some other criteria, such as BIC and RSS. Let's try BIC first.

The BIC gives us the same model as did CP. How about using adjusted $R^2$,

It is interesting that the three criteria gave us the the same overall best model. We can further have a look at the RSS plot.

In [ ]:
plot(reg.summary$rss, xlab = "Number of variables", ylab = "RSS", type = "l")

What we observed is that RSS decrease monotonicity as the number variable increases.

## 1.3 Perform stepwise selection

One drawback of best subset selection is its time complexity. In general, the number of models that you need to fit is $2^p$, which is infeasible if $p$ is large, for example, if $p =10$. It also easily results in a model that fits the training data perfectly but performing badly on unseen data. In this section, we are going to explore alternative methods, known as stepwise selection, which explore a far more restricted set of models. There are three stepwise selection methods:
* <b>Forward stepwise selection</b>: Starts with one-variable models, gradually add one variable, end with a model including all the specified variables.
* <b>Backward stepwise selection</b>: Starts with a full model, gradually exclude one variable, end with one-variable models.
* <b>Hybrid selection</b>: the combination of Forward stepwise selection and Backward stepwise selection.

We begin with forward stepwise selection. It is very easy to implement the forward stepwise selection method with <font color="blue">regsubsets()</font>: 

In [ ]:
regfit.fwd <-  

In [ ]:
reg.summary.fwd <-
reg.summary.fwd 
reg.summary

Have you observed the differences between the above output and that generated by the best subset selection? For example, is the five-variable model generated by the forward stepwise selection the same as the one generated by the best subset selection? 

The answer is No. the five-variable model generated by the forward stepwise selection contains $X$, $X^2$, $X^4$, $X^5$ and $X^9$. However, the one generated by the best subset selection contains $X$, $X^2$, $X^5$, $X^8$ and $X^9$. You can also find that the 6,7,8-variable models are also different.

Now, what is the overall best model? Similarly, we can generate a set of plots to identify the best overall model as follows.

In [ ]:
par(mfrow = c(2, 2))
plot(reg.summary.fwd$cp, xlab = "Number of variables", ylab = "C_p", type = "l")
points(which.min(reg.summary.fwd$cp), reg.summary.fwd$cp[which.min(reg.summary.fwd$cp)], col = "red", cex = 2, pch = 20)
plot(reg.summary.fwd$bic, xlab = "Number of variables", ylab = "BIC", type = "l")
points(which.min(reg.summary.fwd$bic), reg.summary.fwd$bic[which.min(reg.summary.fwd$bic)], col = "red", cex = 2, pch = 20)
plot(reg.summary.fwd$adjr2, xlab = "Number of variables", ylab = "Adjusted R^2", type = "l")
points(which.max(reg.summary.fwd$adjr2), reg.summary.fwd$adjr2[which.max(reg.summary.fwd$adjr2)], col = "red", cex = 2, pch = 20)
plot(reg.summary.fwd$rss, xlab = "Number of variables", ylab = "RSS", type = "l")
mtext("Plots of C_p, BIC, adjusted R^2 and RSS for forward stepwise selection", side = 3, line = -2, outer = TRUE)

The above plots show that the best overall model is still the 3-variable model. How about the coefficients?

In [ ]:
coef(regfit.fwd, 3)

You will find the estimated coefficients are the same.

Now, let's have a look at the backward selection method. To implement it, we just simply, change the value of <font color="orange">method</font> to <b>backward</b>.

In [ ]:
regfit.bwd <- 
reg.summary.bwd <- 
reg.summary.bwd

Can you identify the differences between the models selected by the backward selection method and those selected by the forward selection method?

Similarly, we can find the best overall model given by the backward selection method.

In [ ]:
par(mfrow = c(2, 2))
plot(reg.summary.bwd$cp, xlab = "Number of variables", ylab = "C_p", type = "l")
points(which.min(reg.summary.bwd$cp), reg.summary.bwd$cp[which.min(reg.summary.bwd$cp)], col = "red", cex = 2, pch = 20)
plot(reg.summary.bwd$bic, xlab = "Number of variables", ylab = "BIC", type = "l")
points(which.min(reg.summary.bwd$bic), reg.summary.bwd$bic[which.min(reg.summary.bwd$bic)], col = "red", cex = 2, pch = 20)
plot(reg.summary.bwd$adjr2, xlab = "Number of variables", ylab = "Adjusted R^2", type = "l")
points(which.max(reg.summary.bwd$adjr2), reg.summary.bwd$adjr2[which.max(reg.summary.bwd$adjr2)], col = "red", cex = 2, pch = 20)
plot(reg.summary.bwd$rss, xlab = "Number of variables", ylab = "Adjusted R^2", type = "l")
mtext("Plots of C_p, BIC, adjusted R^2 and RSS for backward stepwise selection", side = 3, line = -2, outer = TRUE)

The best overall model is still the three-variable model with exactly the same variables.

## 1.4 Perform Lasso
What we have done so far is about selecting the best subset of variables used in the model, which is often known as feature selection. As discussed in the lecture, the Lasso is one approach that we often used in machine learning to perform feature selection automatically. Depending on the shrinkage parameter, Lasso regularizes the coefficient in a way such that the estimated coefficients can be shrunk toward zero. The R library that we are going to use is the <font color="orange">glmnet</font>. It's document table can be found <a href="https://cran.r-project.org/web/packages/glmnet/glmnet.pdf">here</a>. Remember that it is always a good idea to refer to the library documentation to check the usage of a given function. The main function that we are going to use is <font color="blue">glmnet()</font>.

In [ ]:
library(glmnet)

In [ ]:
help(glmnet)

Now we are ready to fit a lasso model to the simulated data, again using $X$,$X_2$, $\dots$, $X_{10}$ as predictors. Use cross-validation to select the optimal value of $\lambda$. Create plots of the cross-validation error as a function of $\lambda$. Report the resulting coefficient estimates, and discuss the results obtained.


In [ ]:
xmat <- model.matrix(y ~ x + I(x^2) + I(x^3) + I(x^4) + I(x^5) + I(x^6) + I(x^7) + I(x^8) + I(x^9) + I(x^10), data = data.full)[, -1]

As discussed in section 6.6, the <a href="https://stat.ethz.ch/R-manual/R-devel/library/stats/html/model.matrix.html">model.matrix</a> function produces a data matrix corresponding to the 10 variables and also transforms any qualitative variables into dummy variables. In this case, we don't have qualitative variables. You should note that <font color="blue">glmnet()</font> can only take numerical, quantitative inputs.

Now, we fit a lasso model with cross-validation using the <font color="blue">cv.glmnet()</font> function

In [ ]:
cv.lasso <-

If <font color="orange">alpha</font> is set to 0, a Ridge regression model is going to be fit. The default number of folds used is 10. You can also set the number of folds by specifying the value of <font color="orange">nfolds</font>. To see more detailed usage of cv.glmnet, simply type
```R
help(cv.glmnet)
```

We can plot the MSE as a function of the logarithm of $\lambda$ with error bars.

In [ ]:
plot(cv.lasso)

What is the cross-validated lambda value?

In [ ]:
bestlam <- cv.lasso$lambda.min
bestlam

Now we refit our lasso model using $\lambda$ chosen by cross-validation.

In [ ]:
fit.lasso <- glmnet(xmat, y, alpha = 1)
predict(fit.lasso, s = bestlam, type = "coefficients")[1:11, ]

The above result shows that 6 of the 10 coefficient estimates are exactly zero. So the lasso model with $\lambda$ chosen by cross-validation contains only four variable, besides the intercept. 

## 2. Lasso v.s. Ridge Regression

In machine learning, we alos call Lasso as $L_1$ regularizition and Ridge as $L_2$ regularization. The difference between the two regularization methods lies in how they penalise the estimated parameters of your model. The Ridge regularization will shrink all the estimated parameters twowards zero, but never equal to zero. In contrast, the Lasso regularization will force some of the estimated parameters to be zero. Besides the two regularization methods, there are other regularization methods, such as
* <a href="http://statweb.stanford.edu/~tibs/ftp/sparse-grlasso.pdf">group lasso</a>, which regularize the estimated parameters at a group level. Note you don't need to understand it, we just would like to let you be aware of the other regularization methods that are also used in data analysis. 

In this exercise, you are going to study how the Lasso and Ridge regularization 
affect the perforamnce of linear regression. We will develop a linear model to predict the number of applications received using the other variables in the <font color="orange">College</font> data set. We begin with splitting the data set into a training set and a testing set.



In [ ]:
library(ISLR)
data(College)
set.seed(1)
dim(College)
train = sample(1:dim(College)[1], dim(College)[1] / 2)
test <- -train
College.train <- College[train, ]
College.test <- College[test, ]

Remeber that if you would like to check the specification of a R function, you can alwasy call the <font color="blue">help()</font> function. Here, we randonly select a half of the data records for training the linear model, and the other half to be in the testing data set.

### 2.1 Fit a least square model.
For the comparsion purpose, we first fit a linear model using least squares on the training set, and report the test error obtained.

In [ ]:
fit.lm <- 

In [ ]:
pred.lm <- 

In [ ]:
mean((pred.lm - College.test$Apps)^2)

The means quare error is quite large. 

### 2.2 Reguarlize the coefficients with the $L_2$ regularization
Now, we are going to fit a ridge regression model on the training set, with the shrinkage parameter (or tunning) $\lambda$ chosen by cross-validation. Remember that the training data taken by the <font color="blue">glmnet</font> function (Type <font color="blue">help(glmnet)</font> to see its specification) should be a matrix. So we have to store all the training and testing datasets in two matrices. 

In [ ]:
train.mat <- model.matrix(Apps ~ ., data = College.train)[,-1]
test.mat <- model.matrix(Apps ~ ., data = College.test)[,-1]

Now, we generate a list of lambda values that will be used in cross-validation.

In [ ]:
grid <- 10^seq(4, -2, length = 100)

With the generated list of possible lambda values, we can now fit the lasso model with the cv.glmnet function. Type 
```r
help(cv.glmnet)
```
to see the usage information of this function.

In [ ]:
set.seed(1)# the purpose of fixing the seed of the random number generator is to make the result repeatable.
fit.ridge <- 
cv.ridge <- 
bestlam.ridge <- cv.ridge$lambda.min
bestlam.ridge

In [ ]:
pred.ridge <- predict(fit.ridge, s = bestlam.ridge, newx = test.mat)
mean((pred.ridge - College.test$Apps)^2)

### 2.3 Reguarlize the coefficients with the $L_1$ regularization
In this question, we will fit a lasso model on the training set, with λ chosen by cross-validation. Report the test error obtained, along with the number of non-zero coefficient estimates.


In [ ]:
set.seed(1)# the purpose of fixing the seed of the random number generator is to make the result repeatable.
fit.lasso <- 
cv.lasso <- 
bestlam.lasso <- cv.lasso$lambda.min
bestlam.lasso

In [ ]:
pred.lasso <- 
mean((pred.lasso - College.test$Apps)^2)

In [ ]:
predict(fit.lasso, s = bestlam.lasso, type = "coefficients")[1:17, ]

Now, we compare the three models:

In [ ]:
mean((pred.lm - College.test$Apps)^2)
mean((pred.ridge - College.test$Apps)^2)
mean((pred.lasso - College.test$Apps)^2)

The lowest MSE is drived by the lasso model.


Optional readings:
https://rstudio-pubs-static.s3.amazonaws.com/2897_9220b21cfc0c43a396ff9abf122bb351.html